In [1]:
!pip install pandas
!pip install matplotlib
!pip install keras
!pip install wandb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
df = pd.read_pickle('dataset1.pkl')
df

,InitialCondition,Trajectory,Classification
0,"(-2.0, -1.4471544715447155, -1.099526161982576...","([-2.010936428329985, -2.0217547169615546, -2....",Bounded orbit
1,"(-2.0, -1.5447154471544715, -1.189881698919723...","([-2.0118383750574815, -2.023555345946939, -2....",Bounded orbit
2,"(-2.0, -0.21138211382113803, -0.13653797119674...","([-2.001322141896985, -2.00255775461015, -2.00...",Bounded orbit
3,"(-2.0, 0.5040650406504068, 0.3337940367874922,...","([-1.996616905942439, -1.993143643565462, -1.9...",Bounded orbit
4,"(-2.0, -0.9268292682926829, -0.650604284584492...","([-2.0064555976142824, -2.012810042355882, -2....",Bounded orbit
...,...,...,...
1995,"(-1.5121951219512195, 0.016260162601626327, 0....","([-1.5121251443240515, -1.512037597344692, -1....",Crash into primary 1
1996,"(-1.5121951219512195, -1.8048780487804879, -1....","([-1.5254269519443, -1.5385757534377793, -1.55...",Crash into primary 1
1997,"(-1.5121951219512195, 0.7317073170731709, 0.35...","([-1.508605601099163, -1.5049843456167733, -1....",Crash into primary 1
1998,"(-1.4796747967479673, -1.8373983739837398, -1....","([-1.4931627647983017, -1.506569316351737, -1....",Crash into primary 1


In [3]:
#check on the shape of the dataset, to find out along which dimension I should pad
trajectory_shapes = []

# Iterate over each trajectory and get its shape
for index, row in df.iterrows():
    trajectory = np.array(row['Trajectory'])
    trajectory_shapes.append(trajectory.shape)  # Append the shape to the list

# Now you have a list of the shapes of each trajectory
# You can convert this list to a DataFrame for easy inspection
shapes_df = pd.DataFrame(trajectory_shapes, columns=['Coordinates', 'Timesteps'])
filtered_df = shapes_df[shapes_df['Coordinates'] != 4]
filtered_df

,Coordinates,Timesteps


In [30]:
#for now we exclude the early terminated trajectories
import torch

# Assuming trajectory_tensors is a list of tensors containing the trajectories
trajectory_tensors = [torch.tensor(trajectory, dtype=torch.float) for trajectory in df['Trajectory']]

# Filter trajectories to include only those of length 9999
trajectory_tensors_filtered = [t for t in trajectory_tensors if t.shape[1] == 9999]

# Stack the filtered trajectories into a tensor
padded_trajectories_tensor = torch.stack(trajectory_tensors_filtered)


In [31]:
print(padded_trajectories_tensor.shape)

torch.Size([1981, 4, 9999])


In [57]:
#Model architecture
from torch import nn
import torch

def gaussian_init_(n_units, std=1):
    sampler = torch.distributions.Normal(torch.Tensor([0]), torch.Tensor([std/n_units]))
    Omega = sampler.sample((n_units, n_units))[..., 0]
    return Omega


class encoderNet(nn.Module):
    def __init__(self, m, n, b, ALPHA = 1):
        super(encoderNet, self).__init__()
        self.N = m * n
        self.tanh = nn.Tanh()

        self.fc1 = nn.Linear(self.N, 16*ALPHA)
        self.fc2 = nn.Linear(16*ALPHA, 16*ALPHA)
        self.fc3 = nn.Linear(16*ALPHA, b)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)

    def forward(self, x):
        x = x.view(-1, 1, self.N)
        x = self.tanh(self.fc1(x))
        x = self.tanh(self.fc2(x))
        x = self.fc3(x)

        return x


class decoderNet(nn.Module):
    def __init__(self, m, n, b, ALPHA = 1):
        super(decoderNet, self).__init__()

        self.m = m
        self.n = n
        self.b = b

        self.tanh = nn.Tanh()

        self.fc1 = nn.Linear(b, 16*ALPHA)
        self.fc2 = nn.Linear(16*ALPHA, 16*ALPHA)
        self.fc3 = nn.Linear(16*ALPHA, m*n)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)

    def forward(self, x):
        x = x.view(-1, 1, self.b)
        x = self.tanh(self.fc1(x))
        x = self.tanh(self.fc2(x))
        x = self.tanh(self.fc3(x))
        x = x.view(-1, 1, self.m, self.n)
        return x



class dynamics(nn.Module):
    def __init__(self, b, init_scale):
        super(dynamics, self).__init__()
        self.dynamics = nn.Linear(b, b, bias=False)
        #self.dynamics.weight.data = gaussian_init_(b, std=1)
        #nn.init.xavier_normal(self.dynamics.weight)
        self.dynamics.weight = initialisation_vector
        U, _, V = torch.svd(self.dynamics.weight.data)
        self.dynamics.weight.data = torch.mm(U, V.t()) * init_scale


    def forward(self, x):
        x = self.dynamics(x)
        return x


class dynamics_back(nn.Module):
    def __init__(self, b, omega):
        super(dynamics_back, self).__init__()
        self.dynamics = nn.Linear(b, b, bias=False)
        self.dynamics.weight.data = torch.pinverse(omega.dynamics.weight.data.t())

    def forward(self, x):
        x = self.dynamics(x)
        return x


class koopmanAE(nn.Module):
    def __init__(self, m, n, b, steps, steps_back, alpha = 1, init_scale=1):
        super(koopmanAE, self).__init__()
        self.steps = steps
        self.steps_back = steps_back

        self.encoder = encoderNet(m, n, b, ALPHA = alpha)
        self.dynamics = dynamics(b, init_scale)
        self.backdynamics = dynamics_back(b, self.dynamics)
        self.decoder = decoderNet(m, n, b, ALPHA = alpha)


    def forward(self, x, mode='forward'):
        out = []
        out_back = []
        z = self.encoder(x.contiguous())
        q = z.contiguous()


        if mode == 'forward':
            for _ in range(self.steps):
                q = self.dynamics(q)
                out.append(self.decoder(q))

            out.append(self.decoder(z.contiguous()))
            return out, out_back

        if mode == 'backward':
            for _ in range(self.steps_back):
                q = self.backdynamics(q)
                out_back.append(self.decoder(q))

            out_back.append(self.decoder(z.contiguous()))
            return out, out_back

In [58]:
print(padded_trajectories_tensor.shape[0])

1981


In [59]:
import numpy as np



# Rotate to high-dimensional space for each trajectory separately
num_trajectories = padded_trajectories_tensor.shape[0]

# Rotate to high-dimensional space
Q = np.random.standard_normal((64,4))
Q,_ = np.linalg.qr(Q)

processed_X = []    #processed X is a list of trajectories
for i in range(num_trajectories):   
    result = torch.matmul(padded_trajectories_tensor[i].T.float(), torch.tensor(Q.T).float())
    min_val = torch.min(result)
    max_val = torch.max(result)
    result = 2 * (result - min_val) / (max_val - min_val) - 1  
    processed_X.append(result)



In [60]:
print(processed_X[0].shape, len(processed_X))

torch.Size([9999, 64]) 1981


In [61]:
import torch
import torchvision
from torch import nn
from torch import autograd
from torch import optim
from torchvision import transforms, datasets
from torch.autograd import grad
from timeit import default_timer as timer

import matplotlib.pyplot as plt

import numpy as np


import torch.nn.init as init

def set_seed(seed=0):
    """Set one seed for reproducibility."""
    np.random.seed(seed)
    torch.manual_seed(seed)

def get_device():
    """Get a gpu if available."""
    if torch.cuda.device_count()>0:
        device = torch.device('cuda')
        print("Connected to a GPU")
    else:
        print("Using the CPU")
        device = torch.device('cpu')
    return device


def add_channels(X):
    if len(X.shape) == 2:
        return X.reshape(X.shape[0], 1, X.shape[1],1)

    elif len(X.shape) == 3:
        return X.reshape(X.shape[0], 1, X.shape[1], X.shape[2])

    else:
        return "dimenional error"


def weights_init(m):
    if isinstance(m, nn.Linear):
        if m.weight is not None:
            init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0.0)

In [64]:
#the train method

import torch
from torch import nn
import numpy as np
import wandb



def train(model, train_loader, lr, weight_decay,
          lamb, num_epochs, learning_rate_change, epoch_update,
          nu=0.0, eta=0.0, backward=0, steps=1, steps_back=1, gradclip=1):
        # Initialize wandb project
    wandb.init(project="with larger dimensions of koopman operator", name="step size 10")
    
    # Log hyperparameters
    wandb.config.lr = lr
    wandb.config.weight_decay = weight_decay
    wandb.config.lamb = lamb
    wandb.config.num_epochs = num_epochs
    wandb.config.learning_rate_change = learning_rate_change
    wandb.config.epoch_update = epoch_update
    wandb.config.nu = nu
    wandb.config.eta = eta
    wandb.config.backward = backward
    wandb.config.steps = steps
    wandb.config.steps_back = steps_back
    wandb.config.gradclip = gradclip

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    device = get_device()


    def lr_scheduler(optimizer, epoch, lr_decay_rate=0.8, decayEpoch=[]):
                    """Decay learning rate by a factor of lr_decay_rate every lr_decay_epoch epochs"""
                    if epoch in decayEpoch:
                        for param_group in optimizer.param_groups:
                            param_group['lr'] *= lr_decay_rate
                        return optimizer
                    else:
                        return optimizer




    criterion = nn.MSELoss().to(device)


    epoch_hist = []
    loss_hist = []
    epoch_loss = []


    for epoch in range(num_epochs):
        #print(epoch)
        for batch_idx, data_list in enumerate(train_loader):
            #print("batch index", batch_idx)
            #print("data list total length" , len(data_list))
            #print("first item in data list length", data_list[0].shape)
            model.train()
            out, out_back = model(data_list[0].to(device), mode='forward')



            for k in range(steps):
                if k == 0:
                    loss_fwd = criterion(out[k], data_list[k+1].to(device))
                    #print("loss between", out[k].shape, data_list[k+1].shape)
                else:
                    loss_fwd += criterion(out[k], data_list[k+1].to(device))


            loss_identity = criterion(out[-1], data_list[0].to(device)) * steps


            loss_bwd = 0.0
            loss_consist = 0.0

            loss_bwd = 0.0
            loss_consist = 0.0

            if backward == 1:
                out, out_back = model(data_list[-1].to(device), mode='backward')


                for k in range(steps_back):

                    if k == 0:
                        loss_bwd = criterion(out_back[k], data_list[::-1][k+1].to(device))
                    else:
                        loss_bwd += criterion(out_back[k], data_list[::-1][k+1].to(device))


                A = model.dynamics.dynamics.weight
                B = model.backdynamics.dynamics.weight

                K = A.shape[-1]

                for k in range(1,K+1):
                    As1 = A[:,:k]
                    Bs1 = B[:k,:]
                    As2 = A[:k,:]
                    Bs2 = B[:,:k]

                    Ik = torch.eye(k).float().to(device)

                    if k == 1:
                        loss_consist = (torch.sum((torch.mm(Bs1, As1) - Ik)**2) + \
                                         torch.sum((torch.mm(As2, Bs2) - Ik)**2) ) / (2.0*k)
                    else:
                        loss_consist += (torch.sum((torch.mm(Bs1, As1) - Ik)**2) + \
                                         torch.sum((torch.mm(As2, Bs2)-  Ik)**2) ) / (2.0*k)





#                Ik = torch.eye(K).float().to(device)
#                loss_consist = (torch.sum( (torch.mm(A, B)-Ik )**2)**1 + \
#                                         torch.sum( (torch.mm(B, A)-Ik)**2)**1 )
#



            loss = loss_fwd + lamb * loss_identity +  nu * loss_bwd + eta * loss_consist

            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), gradclip) # gradient clip
            optimizer.step()

        # schedule learning rate decay
        # Log metrics
        wandb.log({"epoch": epoch, "loss": loss.item(), "loss_identity": loss_identity.item(), "loss_forward": loss_fwd.item()})
        lr_scheduler(optimizer, epoch, lr_decay_rate=learning_rate_change, decayEpoch=epoch_update)
        loss_hist.append(loss)
        epoch_loss.append(epoch)


        if (epoch) % 20 == 0:
                print('********** Epoche %s **********' %(epoch+1))

                print("loss identity: ", loss_identity.item())
                if backward == 1:
                    print("loss backward: ", loss_bwd.item())
                    print("loss consistent: ", loss_consist.item())
                print("loss forward: ", loss_fwd.item())
                print("loss sum: ", loss.item())


                epoch_hist.append(epoch+1)

                if hasattr(model.dynamics, 'dynamics'):
                    w, _ = np.linalg.eig(model.dynamics.dynamics.weight.data.cpu().numpy())
                    print(np.abs(w))


    if backward == 1:
        loss_consist = loss_consist.item()


    return model, optimizer, [epoch_hist, loss_fwd.item(), loss_consist]



In [ ]:
from torch.utils.data import DataLoader, Dataset

import torch.nn.init as init

########################################### Model Hyperparameters ##############################################

#Declaring some hyperparameters - default values
lamb = 1
steps = 10
lr =1e-2
bottleneck= 32
lr_update = [30, 200, 400, 500],
epochs = 600
batch = 64
steps_back = 8
wd = 0
lr_decay = 0.2
nu =1e-1
eta = 1e-2
backward = 0
gradclip = 0.05
#pred_steps = 1000
m = 64
n= 1




##############################   TRAIN LOADERS ###########################################################
train_loaders = []
test_dataset_trajectories = []
#should be len(processed_X)
for i in range(100):
    Xtrain = processed_X[i][0:9999]
    Xtest = processed_X[i][6000:10000]
    print("X train length", Xtrain.shape)
    print("X test length", Xtest.shape)
    #test_dataset_trajectories.append(Xtest)
    Xtrain = add_channels(Xtrain)
    Xtest = add_channels(Xtest)
    Xtrain = Xtrain.float().contiguous()
    Xtest = Xtest.float().contiguous()
    trainDat = []
    start = 0
    for i in np.arange(steps,-1, -1):
        if i == 0:
            trainDat.append(Xtrain[start:].float())
        else:
            trainDat.append(Xtrain[start:-i].float())
        start += 1

    train_data = torch.utils.data.TensorDataset(*trainDat)
    del(trainDat)



    train_loader = DataLoader(dataset = train_data,
                              batch_size = batch,
                              shuffle = True)

    train_loaders.append(train_loader)



#==============================================================================
# Model
#==============================================================================

model = koopmanAE(m, n, bottleneck, steps, steps_back)


############################ Printing the Model Architecture  #############################################
#print('koopmanAE')




#model = torch.nn.DataParallel(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)


#==============================================================================
# Model summary
#==============================================================================
'''
print('**** Setup ****')
print('Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))
print('Total params: %.2fk' % (sum(p.numel() for p in model.parameters())/1000.0))
print('************')
print(model)
'''

#==============================================================================
# Start training - training separately on each trajectory
#==============================================================================

# Start training on each trajectory separately
for idx, train_loader in enumerate(train_loaders):
    print(f'Training on trajectory {idx + 1}')
    model, optimizer, epoch_hist = train(model, train_loader, lr, wd, lamb, epochs,
                                         lr_decay, lr_update, nu, eta, backward,
                                         steps, steps_back, gradclip)
    # Save the trained model after training on each trajectory
    torch.save(model.state_dict(), f'model_trained_on_trajectory_{idx + 1}.pkl')



X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size

epoch,▁▂▃▄▅▆▇█
loss,▅█▄▃▆▁▂▅
loss_forward,▆█▄▄▆▁▂▅
loss_identity,▄█▃▂▆▁▁▂
epoch,7
loss,0.18624
loss_forward,0.15293
loss_identity,0.03331


Connected to a GPU
********** Epoche 1 **********
loss identity:  0.0019672978669404984
loss forward:  0.0027005618903785944
loss sum:  0.004667859524488449
[0.998406   0.998406   0.90350646 0.90350646 0.68465996 0.68465996
 0.5688595  0.5688595  0.5852832  0.5852832  0.8076524  0.8076524
 0.7886568  0.7886568  0.9753587  0.9753587  0.47686937 0.47686937
 0.48614737 0.5907529  0.5907529  0.8984576  0.8984576  0.96560603
 0.96560603 0.9874755  0.9874755  0.9570338  0.73725957 0.73725957
 0.8077126  0.82319176]
********** Epoche 21 **********
loss identity:  0.009955357760190964
loss forward:  0.008580353111028671
loss sum:  0.018535710871219635
[0.9993873  0.9993873  0.9957256  0.9957256  0.67297107 0.67297107
 0.7233328  0.7233328  0.571047   0.571047   0.4653129  0.4653129
 0.7480042  0.7480042  0.91732043 0.91732043 0.7847932  0.7847932
 0.8189898  0.8189898  0.5510894  0.5510894  0.84347224 0.84347224
 0.73924553 0.73924553 0.67347234 0.7897073  0.77770525 0.77770525
 0.7524199  0.7

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▆█▄▆▂▄▂▄▇▃▃▃▂▁▂▆▂▂▃▂▃▂▁▂▂▁▂▂▂▂▂▂▂▂▂▁▂▂▆▂
loss_forward,▆█▄▆▃▄▃▄▇▃▃▃▂▁▃▇▂▂▃▂▃▂▁▂▂▂▂▂▂▂▃▂▃▂▂▁▂▂▆▂
loss_identity,▇█▄▆▂▄▂▃▆▃▂▃▂▁▂▅▂▂▃▂▂▂▁▂▂▁▂▂▂▃▂▂▂▂▂▁▂▂▆▂
epoch,599
loss,0.00065
loss_forward,0.00037
loss_identity,0.00028


Connected to a GPU
********** Epoche 1 **********
loss identity:  0.00030960942967794836
loss forward:  0.0003518651647027582
loss sum:  0.0006614745943807065
[0.9994656  0.9994656  1.000179   1.000179   0.8437533  0.8437533
 0.74968076 0.74968076 0.89476675 0.89476675 0.6763941  0.6763941
 0.78173435 0.78173435 0.79870236 0.79870236 0.81145495 0.81145495
 0.6695643  0.6695643  0.78813833 0.78813833 0.7042809  0.7042809
 0.83270234 0.75354064 0.6858527  0.530214   0.530214   0.4647099
 0.53556615 0.53556615]
********** Epoche 21 **********
loss identity:  0.0005124733434058726
loss forward:  0.0007568231085315347
loss sum:  0.0012692965101450682
[0.9987476  0.9987476  1.0003084  0.99785346 0.80376786 0.80376786
 0.78782624 0.78782624 0.63298476 0.63298476 0.7079918  0.7079918
 0.7728128  0.7728128  0.87943137 0.87943137 0.74165255 0.74165255
 0.6032604  0.6032604  0.26118645 0.26118645 0.4108851  0.4108851
 0.77539444 0.77539444 0.7057378  0.6282286  0.6282286  0.48686075
 0.59588414 0

...

Connected to a GPU
********** Epoche 1 **********
loss identity:  0.10664935410022736
loss forward:  0.33830809593200684
loss sum:  0.444957435131073
[62.846405  52.1279    52.1279    37.4536    32.605152  32.605152
 12.358298  12.358298   8.702065   8.702065   8.489792   8.489792
  8.838286   6.636678   6.636678   7.1346116  5.934601   5.450144
  5.450144   4.993071   4.589584   4.589584   4.232955   4.232955
  3.4783528  3.18017    3.4507666  2.2421362  2.2421362  1.4343469
  1.4343469  1.2200353]
********** Epoche 21 **********
loss identity:  0.022220857441425323
loss forward:  0.15428686141967773
loss sum:  0.17650771141052246
[62.81847   52.128143  52.128143  38.109528  32.2391    32.2391
 12.710814  12.710814   9.456669   9.456669   9.181236   8.143649
  8.143649   6.6514955  6.6514955  5.797405   5.3758297  5.3758297
  6.1243114  4.64501    4.64501    3.8942811  3.8942811  3.2508328
  3.2508328  4.7449574  2.5719807  2.5719807  3.3715334  1.2730018
  1.2730018  1.2766753]
*****

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▂▇▂▁▄▂▂▃▂▂▂▂▆▅▃▂▂▂▂▂▅▂▆▂▂▄▂█▂▃▂▂▁▂▃▃▃▁▁▅
loss_forward,▃▇▂▁▄▂▂▃▂▂▂▂▆▅▄▂▂▂▂▂▅▃▅▃▁▄▂█▁▃▂▂▁▂▃▃▄▁▁▅
loss_identity,▂█▁▂▃▂▂▅▂▂▁▂▅▆▃▁▃▁▂▂▅▂█▁▂▄▂▇▂▃▂▂▂▂▂▂▂▂▁▄
epoch,599
loss,0.32658
loss_forward,0.26145
loss_identity,0.06513


Connected to a GPU
********** Epoche 1 **********
loss identity:  0.06343431770801544
loss forward:  0.6214650273323059
loss sum:  0.6848993301391602
[66.26558    54.21379    54.21379    35.423172   30.596573   30.596573
 13.2093725  13.2093725  11.5595045   9.25352     9.25352     8.849199
  8.849199    6.9725327   6.9725327   5.71348     4.6140122   4.6140122
  4.409221    4.409221    4.493466    4.493466    3.3167691   3.3167691
  5.538889    4.491376    2.3772314   2.3772314   2.2180681   2.2180681
  1.5121081   0.07239356]
********** Epoche 21 **********
loss identity:  0.11565794050693512
loss forward:  0.5350528359413147
loss sum:  0.6507107615470886
[66.26558    54.21379    54.21379    35.423172   30.596573   30.596573
 13.2093725  13.2093725  11.5595045   9.25352     9.25352     8.849199
  8.849199    6.9725327   6.9725327   5.71348     4.6140122   4.6140122
  4.409221    4.409221    4.493466    4.493466    3.3167691   3.3167691
  5.538889    4.491376    2.3772314   2.3772314 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▅▄█▅▅▄▄█▁▃▄▂▃▂▅▇▇▅▄▃▄▄▅▇▆▄▄▅▅▄▄▇▄▅▄▂▅▅▆▃
loss_forward,▅▄▇▅▅▅▂▇▁▁▅▃▃▂▆▇▇▅▅▂▃▃▅▆▇▄▃▄▄▄▃█▄▅▅▁▅▄▅▂
loss_identity,▄▄█▄▂▂█▆▃▆▃▂▄▄▁▅▄▃▂▇▆▆▄▆▃▄█▆█▄▆▃▄▄▁▃▅▅▅▄
epoch,599
loss,0.59276
loss_forward,0.49599
loss_identity,0.09677


Connected to a GPU
********** Epoche 1 **********
loss identity:  0.3849482536315918
loss forward:  0.8422215580940247
loss sum:  1.2271697521209717
[66.26558    54.21379    54.21379    35.423172   30.596573   30.596573
 13.2093725  13.2093725  11.5595045   9.25352     9.25352     8.849199
  8.849199    6.9725327   6.9725327   5.71348     4.6140122   4.6140122
  4.409221    4.409221    4.493466    4.493466    3.3167691   3.3167691
  5.538889    4.491376    2.3772314   2.3772314   2.2180681   2.2180681
  1.5121081   0.07239356]
********** Epoche 21 **********
loss identity:  0.029302168637514114
loss forward:  0.7297488451004028
loss sum:  0.7590510249137878
[66.26558    54.21379    54.21379    35.423172   30.596573   30.596573
 13.2093725  13.2093725  11.5595045   9.25352     9.25352     8.849199
  8.849199    6.9725327   6.9725327   5.71348     4.6140122   4.6140122
  4.409221    4.409221    4.493466    4.493466    3.3167691   3.3167691
  5.538889    4.491376    2.3772314   2.3772314 

In [56]:
initialisation_vector = model.dynamics.dynamics.weight
initialisation_vector

Parameter containing:
tensor([[-0.8545,  0.0036, -0.1912,  ..., -0.3051, -0.1084, -0.0723],
        [ 0.0773, -0.7592,  0.1208,  ...,  0.3666, -0.0368,  0.1148],
        [-0.0232,  0.1634, -0.6211,  ..., -0.3069, -0.0231, -0.0563],
        ...,
        [ 0.1740,  0.2514,  0.4879,  ..., -0.3687, -0.0201,  0.1989],
        [-0.1444,  0.0735, -0.2190,  ..., -0.0151, -0.7889, -0.2110],
        [-0.0606,  0.0130, -0.1001,  ..., -0.1660, -0.0484, -0.8142]],
       device='cuda:0', requires_grad=True)